In [55]:
from openai import OpenAI
import os

OPENAI_API_KEY =''
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [56]:
client = OpenAI()


In [57]:
training_file_id = client.files.create(
  file=open('chat_train.jsonl', "rb"),
  purpose="fine-tune"
)

validation_file_id = client.files.create(
  file=open('chat_validation.jsonl', "rb"),
  purpose="fine-tune"
)

In [ ]:
print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")

In [65]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id.id, 
  validation_file=validation_file_id.id,
  model="gpt-3.5-turbo", 
  hyperparameters={
    "n_epochs": 30,
	"batch_size": 10,
	"learning_rate_multiplier": 0.3
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-DpYhbzewXEIBjvr68trZSToD.
Training Response: FineTuningJob(id='ftjob-DpYhbzewXEIBjvr68trZSToD', created_at=1711211392, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=30, batch_size=10, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sAxiCEm1bAULLyaaYVi9r2Jh', result_files=[], status='validating_files', trained_tokens=None, training_file='file-oeoqdmHmyaJ4bShOMmUloyFg', validation_file='file-LKmbVf530T0p311gHBi4qQ3j', user_provided_suffix=None)
Training Status: validating_files


In [73]:
import signal
import datetime


def signal_handler():
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


print(f"Streaming events for the fine-tuning job: {job_id}")

signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id)
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

Streaming events for the fine-tuning job: ftjob-DpYhbzewXEIBjvr68trZSToD
2024-03-23 19:41:13 The job has successfully completed
2024-03-23 19:41:11 New fine-tuned model created: ft:gpt-3.5-turbo-0125:provectus-prosearch::95yid3wp
2024-03-23 19:40:50 Step 261/270: training loss=1.32, validation loss=1.49
2024-03-23 19:40:28 Step 251/270: training loss=1.14, validation loss=1.91
2024-03-23 19:40:06 Step 241/270: training loss=1.35, validation loss=1.49
2024-03-23 19:39:44 Step 231/270: training loss=1.65, validation loss=1.92
2024-03-23 19:39:21 Step 221/270: training loss=1.36, validation loss=1.50
2024-03-23 19:38:59 Step 211/270: training loss=1.16, validation loss=1.94
2024-03-23 19:38:37 Step 201/270: training loss=1.72, validation loss=1.52
2024-03-23 19:38:15 Step 191/270: training loss=1.30, validation loss=1.96
2024-03-23 19:37:55 Step 181/270: training loss=1.31, validation loss=1.54
2024-03-23 19:37:31 Step 171/270: training loss=1.27, validation loss=1.99
2024-03-23 19:37:08 

In [76]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"FT job {job_id} finished with status: {status}")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

FT job ftjob-DpYhbzewXEIBjvr68trZSToD finished with status: succeeded
Found 2 finetune jobs.


In [77]:
# Retrieve the finetuned model
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:provectus-prosearch::95yid3wp


In [78]:
prompt = "What is the focus of Special Publication SEBD0640?"
completion = client.chat.completions.create(
  model=fine_tuned_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Special Publication SEBD0640 provides guidelines for the installation of a Solar battery disconnect.', role='assistant', function_call=None, tool_calls=None)


In [79]:
# !curl https://api.openai.com/v1/chat/completions \
#   -H "Content-Type: application/json" \
#   -H "Authorization: Bearer $OPENAI_API_KEY" \
#   -d '{"model": "ft:gpt-3.5-turbo-0125:axiom-law::95xLgV7f","messages": [{"role": "system","content": "You are a helpful assistant."},{"role": "user","content": "What is the focus of Special Publication SEBD0640?"}]}'

In [81]:
import requests
prompt = "Focus of Engine Manufacturers Association Parts Manual SERP5069 299C?"
endpoint_url = "https://api.openai.com/v1/chat/completions"
api_key = OPENAI_API_KEY

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
payload = {
    "model": fine_tuned_model,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(endpoint_url, headers=headers, json=payload)

if response.status_code == 200:
    response_json = response.json()
    print("Model's response:", response_json)
else:
    print("Error:", response.text)


Model's response: {'id': 'chatcmpl-95z1oanivZCx6oD6dgSHoAgD25gBZ', 'object': 'chat.completion', 'created': 1711213260, 'model': 'ft:gpt-3.5-turbo-0125:provectus-prosearch::95yid3wp', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The focus of Engine Manufacturers Association Parts Manual SERP5069 299C includes information on Caterpillar spark-ignited gas engines.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 32, 'completion_tokens': 27, 'total_tokens': 59}, 'system_fingerprint': 'fp_30c5ea69e1'}


In [ ]:
# LAST without validation
# ft:gpt-3.5-turbo-0125:axiom-law::95woVmrq

# last
# ft:gpt-3.5-turbo-0125:axiom-law::95xLgV7f